In [289]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%pylab inline
import scipy.stats
import statsmodels.formula.api as smf
import statsmodels.api as sm
import urllib
import requests
import zipfile
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
from sklearn.decomposition import PCA
import geopandas as gpd
from sklearn.model_selection import train_test_split

Populating the interactive namespace from numpy and matplotlib


# 1. data preparation

In [129]:
def get_LL84(url):
    data = pd.read_excel(url)
    cols = [x.encode('utf8').replace('\xc2\xb2', '2') for x in data.columns]
    data.columns = cols
    data.rename(columns = {'NYC Borough, Block, and Lot (BBL)':'BBL',
                           'NYC Borough, Block and Lot (BBL)':'BBL','Zip Code':'Zip',
                           'Site EUI (kBtu/ft2)':'Site EUI',
        'Site EUI\n(kBtu/ft2)':'Site EUI','DOF Benchmarking Submission Status':'Benchmarking Submission',
                           'Weather Normalized Source EUI\n(kBtu/ft2)':'Weather Normalized Source EUI',
                           'Weather Normalized Source EUI (kBtu/ft2)':'Weather Normalized Source EUI',
                           'Municipally Supplied Potable Water - Indoor Intensity (gal/ft2)':'Indoor Water Intensity(gal/ft2)',
                           'Indoor Water Intensity (All Water Sources)\n(gal/ft2)':'Indoor Water Intensity(gal/ft2)',
                           'Water per Square Foot':'Indoor Water Intensity(gal/ft2)',
                           'Total GHG Emissions\n(MtCO2e)':'GHG',
                           'Total GHG Emissions (Metric Tons CO2e)':'GHG',
                           'Municipally Supplied Potable Water - Indoor Intensity (gal/ft2)':'Indoor Water Intensity(gal/ft2)',
                               'Property Floor Area (Buildngs and Parking)\n(ft2)':'Property Floor Area(ft2)',
                           'DOF Property Floor Area (ft2)':'Property Floor Area(ft2)',
                               'DOF Property Floor Area (Buildngs and Parking)\n(ft2)':'Property Floor Area(ft2)',
                           'DOF Property Floor Area (ft²)':'Property Floor Area(ft2)',
                               'DOF Number of Buildings':'Number of Buildings',
                           'Number of Buildings - Self-reported':'Number of Buildings',
                          'Primary Property Type - Self Selected':'Reported Facility Type'},inplace=True)
    
    
    data = data[data['ENERGY STAR Score']>0][['BBL','Zip','Benchmarking Submission','Site EUI','Weather Normalized Source EUI',
                 'Indoor Water Intensity(gal/ft2)','Reported Water Method','ENERGY STAR Score'
                ,'GHG','Property Floor Area(ft2)','Reported Facility Type','Number of Buildings']]
    return data

In [132]:
LL13 = get_LL84('http://www.nyc.gov/html/gbee/downloads/excel/2013_nyc_ll84_disclosure.xlsx')

In [133]:
LL14 = get_LL84('http://www.nyc.gov/html/gbee/downloads/excel/150428_2014_nyc_ll84_disclosure.xlsx')

In [134]:
LL15 = get_LL84('http://www.nyc.gov/html/gbee/downloads/excel/2015_nyc_cy2014__ll84_disclosure_data.xlsx')

In [135]:
LL16 = get_LL84('http://www.nyc.gov/html/gbee/downloads/excel/nyc_benchmarking_disclosure_data_reported_in_2016.xlsx')

In [136]:
PRICE = pd.read_csv('Housing_price/2016.csv')

## As we have only 4 years of data here, we would now only use data of 2016 as train data and the others as test data

In [148]:
LL16.BBL = LL16.BBL.astype('str').str[:-2]

In [150]:
merged16 = pd.merge(LL16,PRICE,on='BBL')

In [151]:
merged16 = merged16[merged16.VALUE>0]

In [154]:
merged16.dropna(axis=0,inplace = True)

In [165]:
merged16.columns = [x.replace(' ','_') for x in merged16.columns]

In [169]:
merged16.head()

BBL      Zip Benchmarking_Submission  Site_EUI  \
2    1014270028  10021.0           In Compliance      44.9   
3    1015180024  10128.0           In Compliance      91.3   
49   1000260021  10005.0           In Compliance      63.5   
87   1000520021  10006.0           In Compliance      84.6   
108  1000680028  10038.0           In Compliance     107.7   

     Weather_Normalized_Source_EUI  Indoor_Water_Intensity(gal/ft2)  \
2                            105.2                            71.51   
3                            180.7                            64.86   
49                           128.4                            34.22   
87                           131.1                            23.11   
108                          231.7                            88.05   

    Reported_Water_Method  ENERGY_STAR_Score     GHG  \
2                  Manual               80.0   538.4   
3                     ABS               10.0   699.3   
49                    ABS               39.0  2163.1   
87                    ABS               24.0   379.0   
108                   ABS               40.0   534.2   

     Property_Floor_Area(ft2) Reported_Facility_Type  Number_of_Buildings  \
2                    166432.0    Multifamily Housing                  1.0   
3                    114939.0    Multifamily Housing                  1.0   
49                   493187.0    Multifamily Housing                  1.0   
87                    57945.0    Multifamily Housing                  1.0   
108                   71539.0                  Hotel                  1.0   

         VALUE  
2     33931000  
3     24268000  
49   114278000  
87     6108000  
108   30130000

# 2. PCA

In [239]:
X_num = merged16[list(merged16.columns[3:6])+list(merged16.columns[7:10])+['Number_of_Buildings','VALUE']]

### Normalization

In [249]:
def norm(data,columns):
    for x in columns:
        data[x] = (data[x]-np.mean(data[x]))/np.std(data[x])
    return data

In [247]:
columns = list(merged16.columns[3:6])+list(merged16.columns[7:10])+['Number_of_Buildings','VALUE']

In [253]:
merged16 = norm(merged16,columns)

### Remove outliers

In [254]:
def clean(data,columns):
    for x in columns:
        data = data[(data[x]<np.mean(data[x])+2*np.std(data[x]))&(data[x]>np.mean(data[x])-2*np.std(data[x]))]
    return data

In [256]:
merged16 = clean(merged16,columns)

In [261]:
X_num = merged16[list(merged16.columns[3:6])+list(merged16.columns[7:10])+['Number_of_Buildings']]
pca = PCA(0.95)
Xproj = pca.fit_transform(X_num)

In [262]:
pca.explained_variance_ratio_

array([ 0.84502224,  0.11743693])

In [263]:
pca.components_

array([[  1.71017040e-02,   6.93292655e-03,   4.40833012e-02,
         -9.98357453e-01,   1.64293282e-02,  -2.68535902e-02,
          2.73625129e-03],
       [ -1.37964318e-03,   3.27544465e-04,  -8.07239675e-02,
         -2.86612954e-02,   8.81428118e-02,   9.91195009e-01,
          4.92286466e-02]])

In [264]:
components_explanation = pd.DataFrame(pca.components_,columns = X_num.columns)
components_explanation

Site_EUI  Weather_Normalized_Source_EUI  Indoor_Water_Intensity(gal/ft2)  \
0  0.017102                       0.006933                         0.044083   
1 -0.001380                       0.000328                        -0.080724   

   ENERGY_STAR_Score       GHG  Property_Floor_Area(ft2)  Number_of_Buildings  
0          -0.998357  0.016429                 -0.026854             0.002736  
1          -0.028661  0.088143                  0.991195             0.049229

## Here we could find the impact of each original variable on 2 principle components
## Based on the coefficients, we could explain each component as:
## Component 1: Energy efficiency
## Component 2: Property construction area

# 3. Analysis

In [265]:
merged16['Energy_efficiency'] = Xproj[:,0]
merged16['Property_area'] = Xproj[:,1]

### Hierarchical linear regression ---- to check if energy efficiency actually works

In [282]:
lm_no_energy = smf.ols(formula='VALUE~Property_area+C(Benchmarking_Submission)+C(Reported_Water_Method)+\
                       C(Reported_Facility_Type)-1'
             ,data=merged16).fit()
lm_no_energy.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  VALUE   R-squared:                       0.280
Model:                            OLS   Adj. R-squared:                  0.276
Method:                 Least Squares   F-statistic:                     74.12
Date:                Mon, 20 Nov 2017   Prob (F-statistic):          1.69e-178
Time:                        00:25:45   Log-Likelihood:                 15.961
No. Observations:                2685   AIC:                            -1.923
Df Residuals:                    2670   BIC:                             86.51
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
===========================================================================================================================
                                                              coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------
C(Benchmarking_Submission)[In Compliance]                  -0.3541      0.241     -1.468      0.142      -0.827       0.119
C(Reported_Water_Method)[T.Manual]                          0.0813      0.032      2.507      0.012       0.018       0.145
C(Reported_Facility_Type)[T.Distribution Center]            0.0028      0.258      0.011      0.991      -0.503       0.508
C(Reported_Facility_Type)[T.Financial Office]               0.3022      0.343      0.882      0.378      -0.370       0.974
C(Reported_Facility_Type)[T.Hotel]                          0.4120      0.264      1.559      0.119      -0.106       0.930
C(Reported_Facility_Type)[T.K-12 School]                    0.3892      0.270      1.443      0.149      -0.140       0.918
C(Reported_Facility_Type)[T.Medical Office]                 0.3616      0.264      1.368      0.172      -0.157       0.880
C(Reported_Facility_Type)[T.Multifamily Housing]            0.0418      0.241      0.173      0.863      -0.431       0.515
C(Reported_Facility_Type)[T.Non-Refrigerated Warehouse]     0.0044      0.248      0.018      0.986      -0.482       0.491
C(Reported_Facility_Type)[T.Office]                         0.3810      0.242      1.573      0.116      -0.094       0.856
C(Reported_Facility_Type)[T.Residence Hall/Dormitory]       0.3491      0.249      1.404      0.160      -0.138       0.837
C(Reported_Facility_Type)[T.Retail Store]                   0.0831      0.341      0.244      0.808      -0.586       0.752
C(Reported_Facility_Type)[T.Senior Care Community]          0.3630      0.251      1.445      0.148      -0.129       0.855
C(Reported_Facility_Type)[T.Worship Facility]               0.4687      0.295      1.586      0.113      -0.111       1.048
Property_area                                               0.3765      0.014     26.451      0.000       0.349       0.404
==============================================================================
Omnibus:                      594.401   Durbin-Watson:                   0.614
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1106.936
Skew:                           1.369   Prob(JB):                    4.28e-241
Kurtosis:                       4.548   Cond. No.                         260.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [283]:
lm_with_energy = smf.ols(formula='VALUE~Energy_efficiency+Property_area+C(Benchmarking_Submission)+C(Reported_Water_Method)+\
                       C(Reported_Facility_Type)-1'
             ,data=merged16).fit()
lm_with_energy.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  VALUE   R-squared:                       0.286
Model:                            OLS   Adj. R-squared:                  0.282
Method:                 Least Squares   F-statistic:                     71.33
Date:                Mon, 20 Nov 2017   Prob (F-statistic):          1.40e-182
Time:                        00:26:11   Log-Likelihood:                 27.745
No. Observations:                2685   AIC:                            -23.49
Df Residuals:                    2669   BIC:                             70.84
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
===========================================================================================================================
                                                              coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------
C(Benchmarking_Submission)[In Compliance]                  -0.3617      0.240     -1.505      0.132      -0.833       0.109
C(Reported_Water_Method)[T.Manual]                          0.0793      0.032      2.456      0.014       0.016       0.143
C(Reported_Facility_Type)[T.Distribution Center]            0.0205      0.257      0.080      0.936      -0.483       0.524
C(Reported_Facility_Type)[T.Financial Office]               0.2783      0.341      0.815      0.415      -0.391       0.948
C(Reported_Facility_Type)[T.Hotel]                          0.4225      0.263      1.606      0.109      -0.094       0.938
C(Reported_Facility_Type)[T.K-12 School]                    0.3962      0.269      1.475      0.140      -0.130       0.923
C(Reported_Facility_Type)[T.Medical Office]                 0.3552      0.263      1.349      0.177      -0.161       0.871
C(Reported_Facility_Type)[T.Multifamily Housing]            0.0500      0.240      0.208      0.835      -0.421       0.521
C(Reported_Facility_Type)[T.Non-Refrigerated Warehouse]     0.0150      0.247      0.061      0.952      -0.470       0.500
C(Reported_Facility_Type)[T.Office]                         0.3752      0.241      1.556      0.120      -0.098       0.848
C(Reported_Facility_Type)[T.Residence Hall/Dormitory]       0.3666      0.248      1.480      0.139      -0.119       0.852
C(Reported_Facility_Type)[T.Retail Store]                   0.0688      0.340      0.202      0.840      -0.597       0.735
C(Reported_Facility_Type)[T.Senior Care Community]          0.3574      0.250      1.429      0.153      -0.133       0.848
C(Reported_Facility_Type)[T.Worship Facility]               0.4615      0.294      1.569      0.117      -0.115       1.038
Energy_efficiency                                          -0.0259      0.005     -4.851      0.000      -0.036      -0.015
Property_area                                               0.3768      0.014     26.584      0.000       0.349       0.405
==============================================================================
Omnibus:                      595.323   Durbin-Watson:                   0.633
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1114.183
Skew:                           1.366   Prob(JB):                    1.14e-242
Kurtosis:                       4.579   Cond. No.                         260.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [284]:
anova = sm.stats.anova_lm(lm_no_energy,lm_with_energy)
anova

df_resid         ssr  df_diff   ss_diff          F    Pr(>F)
0    2670.0  155.348282      0.0       NaN        NaN       NaN
1    2669.0  153.990696      1.0  1.357586  23.529974  0.000001

In [285]:
anova['Pr(>F)'][1] < 0.05

True

## As p here is way less than our significance level 5%, We could conclude that energy efficiency actually has a significant impact on property market value

### Linear Regression
### Optimize the model using feature selection (maximizing adj-R2)

In [266]:
merged16.head()

BBL      Zip Benchmarking_Submission  Site_EUI  \
3    1015180024  10128.0           In Compliance -0.017825   
87   1000520021  10006.0           In Compliance -0.023664   
118  1000710001  10038.0           In Compliance -0.013903   
121  1000720027  10038.0           In Compliance -0.066195   
126  1000760024  10038.0           In Compliance -0.041444   

     Weather_Normalized_Source_EUI  Indoor_Water_Intensity(gal/ft2)  \
3                        -0.008997                        -0.020404   
87                       -0.022774                        -0.166633   
118                      -0.017135                        -0.121766   
121                      -0.040216                        -0.152868   
126                      -0.026301                        -0.159663   

    Reported_Water_Method  ENERGY_STAR_Score       GHG  \
3                     ABS          -1.871790 -0.012300   
87                    ABS          -1.360822 -0.072823   
118                   ABS          -1.433817 -0.062052   
121                   ABS           1.376507 -0.121422   
126                   ABS           0.938534 -0.092776   

     Property_Floor_Area(ft2) Reported_Facility_Type  Number_of_Buildings  \
3                    0.053541    Multifamily Housing            -0.088747   
87                  -0.561290    Multifamily Housing            -0.088747   
118                 -0.614969    Multifamily Housing            -0.088747   
121                 -0.625369    Multifamily Housing            -0.088747   
126                 -0.517244    Multifamily Housing            -0.088747   

        VALUE  Energy_efficiency  Property_area  
3    0.530914           1.994778       0.399601  
87  -0.348515           1.493524      -0.217988  
118 -0.046381           1.570202      -0.271787  
121 -0.402027          -1.238628      -0.365300  
126  0.225341          -0.803587      -0.242531

In [290]:
train,test = train_test_split(merged16, test_size = 0.3)

In [324]:
lm1 = smf.ols(formula='VALUE~Energy_efficiency+Property_area+C(Benchmarking_Submission)+C(Reported_Water_Method)+C(Reported_Facility_Type)'
             ,data=train).fit()
lm1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  VALUE   R-squared:                       0.280
Model:                            OLS   Adj. R-squared:                  0.275
Method:                 Least Squares   F-statistic:                     48.41
Date:                Mon, 20 Nov 2017   Prob (F-statistic):          1.99e-121
Time:                        01:04:01   Log-Likelihood:                 17.215
No. Observations:                1879   AIC:                            -2.429
Df Residuals:                    1863   BIC:                             86.19
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
===========================================================================================================================
                                                              coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------
Intercept                                                  -0.3631      0.241     -1.508      0.132      -0.835       0.109
C(Reported_Water_Method)[T.Manual]                          0.0643      0.038      1.682      0.093      -0.011       0.139
C(Reported_Facility_Type)[T.Distribution Center]            0.0270      0.260      0.104      0.917      -0.483       0.537
C(Reported_Facility_Type)[T.Financial Office]               0.3022      0.343      0.881      0.378      -0.370       0.975
C(Reported_Facility_Type)[T.Hotel]                          0.4648      0.278      1.672      0.095      -0.081       1.010
C(Reported_Facility_Type)[T.K-12 School]                    0.4439      0.278      1.596      0.111      -0.101       0.989
C(Reported_Facility_Type)[T.Medical Office]                 0.4129      0.278      1.483      0.138      -0.133       0.959
C(Reported_Facility_Type)[T.Multifamily Housing]            0.0537      0.241      0.223      0.824      -0.419       0.526
C(Reported_Facility_Type)[T.Non-Refrigerated Warehouse]    -0.0048      0.250     -0.019      0.985      -0.495       0.486
C(Reported_Facility_Type)[T.Office]                         0.3688      0.242      1.523      0.128      -0.106       0.844
C(Reported_Facility_Type)[T.Residence Hall/Dormitory]       0.3876      0.253      1.535      0.125      -0.108       0.883
C(Reported_Facility_Type)[T.Retail Store]                   0.0732      0.341      0.215      0.830      -0.595       0.741
C(Reported_Facility_Type)[T.Senior Care Community]          0.3556      0.253      1.407      0.160      -0.140       0.851
C(Reported_Facility_Type)[T.Worship Facility]               0.4628      0.295      1.569      0.117      -0.116       1.041
Energy_efficiency                                          -0.0220      0.006     -3.446      0.001      -0.035      -0.009
Property_area                                               0.3666      0.017     21.958      0.000       0.334       0.399
==============================================================================
Omnibus:                      385.470   Durbin-Watson:                   1.989
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              675.859
Skew:                           1.305   Prob(JB):                    1.73e-147
Kurtosis:                       4.351   Cond. No.                         217.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [325]:
pvalues = lm1.pvalues

In [326]:
pvalues.sort_values(ascending=True)

Property_area                                              3.130896e-95
Energy_efficiency                                          5.817321e-04
C(Reported_Water_Method)[T.Manual]                         9.281261e-02
C(Reported_Facility_Type)[T.Hotel]                         9.475852e-02
C(Reported_Facility_Type)[T.K-12 School]                   1.105920e-01
C(Reported_Facility_Type)[T.Worship Facility]              1.167370e-01
C(Reported_Facility_Type)[T.Residence Hall/Dormitory]      1.250401e-01
C(Reported_Facility_Type)[T.Office]                        1.278746e-01
Intercept                                                  1.318176e-01
C(Reported_Facility_Type)[T.Medical Office]                1.381797e-01
C(Reported_Facility_Type)[T.Senior Care Community]         1.595013e-01
C(Reported_Facility_Type)[T.Financial Office]              3.781726e-01
C(Reported_Facility_Type)[T.Multifamily Housing]           8.235968e-01
C(Reported_Facility_Type)[T.Retail Store]                  8.298

In [322]:
def AdjR2(features):
    lm = smf.ols(formula = ('VALUE~'+features), data = train).fit()
    lmy = lm.predict(test)
    y_err = lmy-test.VALUE
    y_norm = test.VALUE-np.mean(test.VALUE)
    # Adjusted R^2
    R2 = 1 - y_err.dot(y_err) / y_norm.dot(y_norm) * (len(merged16.index)-1) / (len(merged16.index)-(features.count('+')+1+features.count('-'))-1)
    return R2, lm

In [323]:
AdjR2('Energy_efficiency+Property_area+C(Benchmarking_Submission)+C(Reported_Water_Method)+C(Reported_Facility_Type)-1')

(0.29484652625565744,
 <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x1ac99320>)

In [328]:
AdjR2('Property_area')[0]

0.20542377904204556

In [329]:
AdjR2('Energy_efficiency+Property_area')[0]

0.22337348360059217

In [331]:
AdjR2('Energy_efficiency+Property_area+C(Reported_Water_Method)')[0]

0.22615508754401314

In [332]:
AdjR2('Energy_efficiency+Property_area+C(Reported_Water_Method)+C(Reported_Facility_Type)')[0]

0.29537276019128755

In [333]:
AdjR2('Energy_efficiency+Property_area+C(Reported_Water_Method)+C(Reported_Facility_Type)-1')[0]

0.29510974143809277

In [334]:
lm = smf.ols(formula='VALUE~Energy_efficiency+Property_area+C(Reported_Water_Method)+C(Reported_Facility_Type)'
             ,data=train).fit()
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  VALUE   R-squared:                       0.280
Model:                            OLS   Adj. R-squared:                  0.275
Method:                 Least Squares   F-statistic:                     48.41
Date:                Mon, 20 Nov 2017   Prob (F-statistic):          1.99e-121
Time:                        01:07:59   Log-Likelihood:                 17.215
No. Observations:                1879   AIC:                            -2.429
Df Residuals:                    1863   BIC:                             86.19
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
===========================================================================================================================
                                                              coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------
Intercept                                                  -0.3631      0.241     -1.508      0.132      -0.835       0.109
C(Reported_Water_Method)[T.Manual]                          0.0643      0.038      1.682      0.093      -0.011       0.139
C(Reported_Facility_Type)[T.Distribution Center]            0.0270      0.260      0.104      0.917      -0.483       0.537
C(Reported_Facility_Type)[T.Financial Office]               0.3022      0.343      0.881      0.378      -0.370       0.975
C(Reported_Facility_Type)[T.Hotel]                          0.4648      0.278      1.672      0.095      -0.081       1.010
C(Reported_Facility_Type)[T.K-12 School]                    0.4439      0.278      1.596      0.111      -0.101       0.989
C(Reported_Facility_Type)[T.Medical Office]                 0.4129      0.278      1.483      0.138      -0.133       0.959
C(Reported_Facility_Type)[T.Multifamily Housing]            0.0537      0.241      0.223      0.824      -0.419       0.526
C(Reported_Facility_Type)[T.Non-Refrigerated Warehouse]    -0.0048      0.250     -0.019      0.985      -0.495       0.486
C(Reported_Facility_Type)[T.Office]                         0.3688      0.242      1.523      0.128      -0.106       0.844
C(Reported_Facility_Type)[T.Residence Hall/Dormitory]       0.3876      0.253      1.535      0.125      -0.108       0.883
C(Reported_Facility_Type)[T.Retail Store]                   0.0732      0.341      0.215      0.830      -0.595       0.741
C(Reported_Facility_Type)[T.Senior Care Community]          0.3556      0.253      1.407      0.160      -0.140       0.851
C(Reported_Facility_Type)[T.Worship Facility]               0.4628      0.295      1.569      0.117      -0.116       1.041
Energy_efficiency                                          -0.0220      0.006     -3.446      0.001      -0.035      -0.009
Property_area                                               0.3666      0.017     21.958      0.000       0.334       0.399
==============================================================================
Omnibus:                      385.470   Durbin-Watson:                   1.989
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              675.859
Skew:                           1.305   Prob(JB):                    1.73e-147
Kurtosis:                       4.351   Cond. No.                         217.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Log-log Linear Regression